In [1]:
import time
import pandas as pd
import numpy as np
import joblib
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

2023-07-10 17:51:37.438135: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-10 17:51:37.617885: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-10 17:51:37.620281: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 17:51:38.776445: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv("GPT-wiki-intro.csv")
data = data[:80000]
intro = []
target = []

for i in data["wiki_intro"]:
    intro.append(i)
    target.append(0)
for j in data["generated_intro"]:
    intro.append(j)
    target.append(1)

intros = np.array(intro)
targets = np.array(target)

In [8]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(intros, targets, test_size=0.2, random_state=42)

# Charger le vectorizer sauvegardé
vectorizer = joblib.load("vectorizer")

# Appliquer le vectorizer sur les données de train et de test
X_train_preprocessed = vectorizer.transform(X_train)
X_test_preprocessed = vectorizer.transform(X_test)

In [6]:
# Créer le modèle de classification avec TensorFlow
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train_preprocessed.shape[1]))
model.add(Dense(1, activation='sigmoid'))

# Compiler le modèle
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])


In [7]:
# Entraîner le modèle en utilisant le GPU si disponible
with tf.device('/GPU:0'):
    model.fit(X_train_preprocessed, y_train, epochs=2, batch_size=32)


Epoch 1/2
4000/4000 [==============================] - 1019s 254ms/step - loss: 0.1730 - accuracy: 0.9333
Epoch 2/2
4000/4000 [==============================] - 1189s 297ms/step - loss: 0.0387 - accuracy: 0.9863


In [11]:
# Évaluer le modèle sur les données de test
loss, accuracy = model.evaluate(X_test_preprocessed, y_test)

print("Précision sur les données de test:", accuracy)

1000/1000 [==============================] - 3s 3ms/step - loss: 0.2333 - accuracy: 0.9103
Précision sur les données de test: 0.910281240940094


In [12]:
model.save("tensorflow_model")

INFO:tensorflow:Assets written to: tensorflow_model/assets


INFO:tensorflow:Assets written to: tensorflow_model/assets


In [54]:
# Charger le vectorizer sauvegardé
vectorizer = joblib.load("vectorizer")
loaded_model = tf.keras.models.load_model("tensorflow_model")

In [72]:
test = """ 
Clive Owen, an accomplished English actor born on October 3, 1964, has garnered recognition for his remarkable performances. Nominated three times for the BAFTA Award for Best Actor, Owen left an indelible mark on the film industry with his roles in "The Wedding Singer" (1998), "Sweet and Lowdown" (1999), and "Mean Streets" (2000). Hailing from Huddersfield, West Yorkshire, Owen's journey began as a mill clerk before he ventured into acting. His illustrious career commenced on the stage and then transitioned to the small screen with the television series "Berkeley Square" (1987). Owen's breakthrough in the 1990s came with notable films like "The Winslow Boy" (1994), "Kes" (1995), and "The Power of One" (1999). Since then, he has left an indelible mark with critically acclaimed performances in films such as "The Artist" (2011), where he received the Academy Award for Best Actor, "Midnight in Paris" (2011), "The Grand Budapest Hotel" (2014), "Nocturnal Animals" (2016), and his compelling role in the war film "Dunkirk" (2017)."""

In [73]:
test_process = vectorizer.transform([test])

score  = loaded_model.predict(test_process)
probabilities = tf.nn.softmax(score)

1/1 [==============================] - 0s 15ms/step


In [74]:
#probabilities[0]

In [75]:
print("gpt",score[0][0])

gpt 0.054063804


In [76]:
print("humain",1-score[0][0])

humain 0.9459361955523491


In [45]:
test_data = data = pd.read_csv("GPT-wiki-intro.csv")[85000:90000]

In [46]:
test_data["wiki_intro"][85001]

"Tibor Feheregyhazi, CM (February 14, 1932, Budapest – July 10, 2007, Saskatoon, Saskatchewan) was a Hungarian-Canadian actor and director. He attended the National Theatre School in Montreal, Quebec. Feheregyhazi went on to become a resident director at the Manitoba Theatre Centre in Winnipeg, Manitoba, as well as artistic director of Magnus Theatre in Thunder Bay, Ontario and lastly, artistic director of the Persephone Theatre in Saskatoon, Saskatchewan. Background\nTibor Feheregyhazi grew up in Budapest. Feheregyhazi's parents worked for the Hungarian National Radio. His mother was executive producer of music programming and his father was an economic journalist. When Feheregyhazi was 4 years old, he had his first radio performance at the station where his parents were employed. He attended a private Jesuit school in Danube before switching to a public high school. He enrolled at the National Hungarian Film and Theatre Academy and acted on stage and radio while at school. He graduat

In [47]:
test_data["generated_intro"][85053]

"B. Kalyanasundaram (born 12 August 1947) is an Indian civil servant and diplomat who has been Secretary (External Affairs) in the Cabinet of India since May 2014. He served as the Permanent Representative of India to the United Nations from 2005 to 2014.\n\nKalyanasundaram was born in Palakkad, Kerala, on 12 August 1947. He graduated with a Bachelor of Arts degree from the University of Calcutta in 1967. He then studied at the University of Sussex, where he completed his Master of Arts degree in 1971.\n\nKalyanasundaram joined the Indian Foreign Service in 1971. He has served as an Ambassador (to Djibouti and Armenia) and a High Commissioner (to Bangladesh and Sri Lanka). From 2005 to 2014, he served as the Permanent Representative of India to the United Nations.\n\nIn January 2014, Kalyanasundaram was appointed Secretary (External Affairs) in the Cabinet of India. In this role, he is responsible for overseeing India's relations with foreign countries and international organisations."